**This is a very basic approach to ANN **



Question:

Based on the data, build a model to predict when the customer will exit the bank. Split the data into Train and Test dataset (70:30), build the model on Train data-set and test the model on Test-dataset. Secondly provide recommendations to the bank so that they can retain the customers who are on the verge of exiting.

**Data dictionary**


RowNumber

CustomerId:Bank ID of the Customer

Surname: Customer’s Surname

CreditScore	: Current Credit score of the customer

Geography:Current country of the customer

Gender:Customer’s Gender

Age:Customer’s Age

Tenure:Customer’s duration association with bank in years	

Balance:Current balance in the bank account.

NumOfProducts

HasCrCard:1 denotes customer has a credit card and 0 denotes customer does not have a credit card	

IsActiveMember:1 denotes customer is an active member and 0 denotes customer is not an active member

Estimated:Customer’s approx. salary

Salary:Salary of Customer

Exited:Target Variable(1 denotes customer has exited the bank and 0 denotes otherwise)


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

We will Start With EDA

In [ ]:
deep=pd.read_csv('../input/deep-learning-az-ann/Churn_Modelling.csv')

In [ ]:
deep.head(10)

In [ ]:
deep.info()

In [ ]:
deep = deep.drop(['RowNumber','CustomerId','Surname'], axis=1)


In [ ]:
deep.head(10)

In [ ]:
deep.describe().T

In [ ]:
deep.nunique()

In [ ]:
deep.Tenure.value_counts()

In [ ]:
deep.Geography.value_counts()

In [ ]:
deep['Exited'].value_counts(normalize=True)

In [ ]:
deep.isnull().sum()

In [ ]:
for i in deep.columns:
    if (deep[i].dtype=='object'):
        print(i)
        print(deep[i].value_counts())
        print('\n')
   

In [ ]:
for i in deep.columns:
    if (deep[i].dtype!='object'):
        print(i)
        print(deep[i].value_counts())
        print('\n')

In [ ]:
for i in deep.columns:
    if(deep[i].dtype=='object'):
        deep[i]=pd.Categorical(deep[i]).codes

In [ ]:
deep.info()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
plt.figure(figsize=(15,15))
deep[['Age','Balance','CreditScore', 'Tenure', 'EstimatedSalary']].boxplot(vert=0)

In [ ]:
sns.pairplot(deep)

In [ ]:
# construct heatmap with only continuous variables
plt.figure(figsize=(10,8))
sns.set(font_scale=1.2)
sns.heatmap(deep[['Age','Balance','CreditScore', 'Tenure', 'EstimatedSalary']].corr(), annot=True)

**We have checked the date for missing values,outliers and other EDA.We can go ahead with the train and test of ANN model.**

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X=deep.drop(['Exited'],axis=1)

In [ ]:
Y=deep['Exited']

In [ ]:
#split data into 70% training and 30% test data
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size= 0.30, random_state=0)

In [ ]:
# Checking dimensions on the train and test data
print('X_train: ',X_train.shape)
print('X_test: ',X_test.shape)
print('Y_train: ',Y_train.shape)
print('Y_test: ',Y_test.shape)

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
#Initialize an object for StandardScaler
sc = StandardScaler()

In [ ]:
#Scale the training data
X_train = sc.fit_transform(X_train)

In [ ]:
X_train

In [ ]:
X_test = sc.transform(X_test)

In [ ]:
X_test

In [ ]:
from sklearn.neural_network import MLPClassifier

In [ ]:
clf = MLPClassifier(hidden_layer_sizes=100, max_iter=5000,
                     solver='sgd', verbose=True,  random_state=21,tol=0.001)

In [ ]:
# Fit the model on the training data
clf.fit(X_train, Y_train)

In [ ]:
y_pred = clf.predict(X_train)

In [ ]:
y_pred

In [ ]:
from sklearn.metrics import confusion_matrix,classification_report

In [ ]:
confusion_matrix(Y_train,y_pred)

In [ ]:
clf.score(X_train,Y_train)

In [ ]:
print(classification_report(Y_train, y_pred))

In [ ]:
# AUC and ROC for the training data
# predict probabilities
probs = clf.predict_proba(X_train)
# keep probabilities for the positive outcome only
probs = probs[:, 1]
# calculate AUC
from sklearn.metrics import roc_auc_score
auc = roc_auc_score(Y_train, probs)
print('AUC: %.3f' % auc)
# calculate roc curve
from sklearn.metrics import roc_curve
fpr, tpr, thresholds = roc_curve(Y_train, probs)
plt.plot([0, 1], [0, 1], linestyle='--')
# plot the roc curve for the model
plt.plot(fpr, tpr, marker='.')
# show the plot
plt.show()

In [ ]:
y_pred_test = clf.predict(X_test)

In [ ]:
clf.score(X_test,Y_test)

In [ ]:
confusion_matrix(Y_test, y_pred_test)

In [ ]:
print(classification_report(Y_test, y_pred_test))

In [ ]:
# AUC and ROC for the test data

# predict probabilities
probs = clf.predict_proba(X_test)
# keep probabilities for the positive outcome only
probs = probs[:, 1]
# calculate AUC
auc = roc_auc_score(Y_test, probs)
print('AUC: %.3f' % auc)
# calculate roc curve
fpr, tpr, thresholds = roc_curve(Y_test, probs)
plt.plot([0, 1], [0, 1], linestyle='--')
# plot the roc curve for the model
plt.plot(fpr, tpr, marker='.')
# show the plot
plt.show()

**We can Use GRID SEARCH to tune the model.**

In [ ]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    'hidden_layer_sizes': [100,200,300,500],
    'max_iter': [5000,2500,7000,6000],
    'solver': ['sgd','adam'],
    'tol': [0.01],
}

nncl = MLPClassifier(random_state=1)

grid_search = GridSearchCV(estimator = nncl, param_grid = param_grid, cv = 10)

In [ ]:
grid_search.fit(X_train, Y_train)

In [ ]:
grid_search.best_params_

In [ ]:
best_grid=grid_search.best_estimator_

In [ ]:
ytrain_predict = best_grid.predict(X_train)
ytest_predict = best_grid.predict(X_test)

In [ ]:
confusion_matrix(Y_train,ytrain_predict)

In [ ]:
clf.score(X_train,Y_train)

In [ ]:
print(classification_report(Y_train,ytrain_predict))

In [ ]:
#from sklearn.metrics import roc_curve,roc_auc_score
rf_fpr, rf_tpr,_=roc_curve(Y_train,best_grid.predict_proba(X_train)[:,1])
plt.plot(rf_fpr,rf_tpr, marker='x', label='NN')
plt.plot(np.arange(0,1.1,0.1),np.arange(0,1.1,0.1))
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC')
plt.show()
print('Area under Curve is', roc_auc_score(Y_train,best_grid.predict_proba(X_train)[:,1]))

In [ ]:
confusion_matrix(Y_test,ytest_predict)

In [ ]:
clf.score(X_test,Y_test)

In [ ]:
print(classification_report(Y_test,ytest_predict))

In [ ]:
clf.score(X_test,Y_test)

In [ ]:
#from sklearn.metrics import roc_curve,roc_auc_score
rf_fpr, rf_tpr,_=roc_curve(Y_test,best_grid.predict_proba(X_test)[:,1])
plt.plot(rf_fpr,rf_tpr, marker='x', label='NN')
plt.plot(np.arange(0,1.1,0.1),np.arange(0,1.1,0.1))
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC')
plt.show()
print('Area under Curve is', roc_auc_score(Y_test,best_grid.predict_proba(X_test)[:,1]))

**The accuracy of the train and test data is approximately equal after grid search and that is approximately 86%.

We can tune the model more if we want to by using the dropped variables and check their affect on data.**

Do upvote thanks :)